# Pokemon Infinite Fusion Optimizer

A fusion optimizer created for the game Pokemon Infinite Fusion

Program created to simplify the process of finding the best fusion for a role, it lets you weight the stats, filter by name, current progress in the playthrough, total base stat, find the best partner for a pokemon and even adjust the importance of the typing, both offensively and defensively.

In [110]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import numpy as np
from itertools import combinations
import os

# PARAMETERS

# Weightings of HP, Attack, Defense, Sp. Atk, Sp. Def, Speed, then offensive and defensive typing scores
# The weight the offensive and defensive typing scores receive directly affect the importance of the typing and the stats, consider them as percentages.
profiles = {
    'Fast physical attacker': ([1, 10, 1, 1, 1, 10], 0.3, 0.01),
    'Fast special attacker': ([1, 1, 1, 10, 1, 10], 0.3, 0.01),
    'Physical tank': ([8, 2, 8, 1, 4, 1], 0.01, 0.25),
    'Special tank': ([8, 1, 4, 2, 8, 1], 0.01, 0.35),
    'Hybrid tank': ([7, 1, 6, 2, 7, 1], 0.01, 0.35),
    'Balanced overall': ([3, 5, 3, 6, 3, 4], 0.2, 0.15),
    "Physical wallbreaker": ([5, 9, 6, 1, 2, 1], 0.2, 0.15),
    "Special wallbreaker": ([5, 1, 2, 9, 6, 1], 0.15, 0.2)
}

# Filter activation
filter_by_pokemon = True
filter_by_medals = True
force_partner = False

# Filter settings
stat_cap = 600
unwanteds = ['Deoxys', 'Blissey', 'Shuckle', 'Chansey', 'Latios', 'Latias', 'Darkrai', 'Wobbuffet', 'Ninjask', 'Ferrothorn', 'Gyarados']
medals = 2
partner = 'Diglett'

# Boosted types
boosted_types = ['fire', 'fighting', 'ground', 'water', 'dark']
boost = 1.5
# Nerfed types
nerfed_types = ['rock', 'bug', 'steel', 'grass']
nerf = 0.75
# Physical types
physical_types = ['normal', 'fighting', 'flying', 'poison', 'ground', 'rock', 'bug', 'steel']
physical_boost = 1
# Special types
special_types = ['fire', 'water', 'grass', 'electric', 'psychic', 'ice', 'dragon', 'fairy']
special_boost = 1

Dataset extraction

In [111]:
def extract_table(url, class_name):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table', {'class': class_name})

    data = []
    for row in table.find_all('tr'):
        row_data = []
        for cell in row.find_all('td'):
            row_data.append(cell.text.strip())
        data.append(row_data)
    return  pd.DataFrame([row for row in data if row != []])


infinite_mons = extract_table("https://infinitefusion.fandom.com/wiki/Pok%C3%A9dex", 'article-table')
infinite_mons = infinite_mons.iloc[:, :2]
infinite_mons.columns = ['Index', 'Name']

pokemon_set = extract_table("https://pokemon-index.com/base", 'move-lev by-base')
pokemon_set = pokemon_set.drop(columns=[0])
pokemon_set['Index'] = pokemon_set.index

Preprocessing

In [112]:
type_dict = {'nor': 'normal', 'fig': 'fighting', 'fly': 'flying', 'poi': 'poison', 'gro': 'ground', 'roc': 'rock', 'bug': 'bug', 'gho': 'ghost', 'ste': 'steel', 'fir': 'fire', 'wat': 'water', 'gra': 'grass', 'ele': 'electric', 'psy': 'psychic', 'ice': 'ice', 'dra': 'dragon', 'dar': 'dark', 'fai': 'fairy'}

pokemon_set['Type 1'] = ''; pokemon_set['Type 2'] = ''
pokemon_set['Type 1'] = pokemon_set[2].str.split(' ').str[0].map(type_dict)
pokemon_set['Type 2'] = pokemon_set[2].str.split(' ').str[1].map(type_dict)

pokemon_set = pokemon_set.drop(columns=[2])
pokemon_set.columns = ['Name', 'HP', 'Attack', 'Defense', 'Special Attack', 'Special Defense', 'Speed', 'Total', 'Index', 'Type 1', 'Type 2']

Fusion of datasets and more preprocessing

In [113]:
pokemon_set = pd.merge(infinite_mons, pokemon_set, on='Name', how='right')
pokemon_set = pokemon_set.loc[((pokemon_set['Index_y'] < 252) & (pokemon_set['Index_y'].notnull())) | ((pokemon_set['Index_x'].notnull()) & (pokemon_set['Index_y'].notnull()))]

pokemon_set.drop(columns=['Index_x'], inplace=True)
pokemon_set.rename(columns={'Index_y': 'Index'}, inplace=True)
for col in ['HP', 'Attack', 'Defense', 'Special Attack', 'Special Defense', 'Speed', 'Total']: pokemon_set[col] = pokemon_set[col].astype(int)

Infinite fusion exceptions

In [114]:
# Exceptions that have their secondary type removed when fusing
exceptions = ['Gyarados', 'Steelix', 'Dragonite', 'Moltres', 'Zapdos', 'Articuno', 'Scyther', 'Gengar', 'Haunter', 'Gastly', 'Onix', 'Golem', 'Graveler', 'Geodude', 'Charizard', 'Venusaur', 'Ivysaur', 'Bulbasaur']

pokemon_set.loc[pokemon_set['Name'].isin(exceptions), 'Type 2'] = np.nan

pokemon_set.loc[(pokemon_set['Type 1'] == 'normal') & (pokemon_set['Type 2'] == 'flying'), 'Type 1'] = 'flying'
pokemon_set.loc[(pokemon_set['Type 1'] == 'normal') & (pokemon_set['Type 2'] == 'flying'), 'Type 2'] = np.nan
pokemon_set.loc[pokemon_set['Name'] == 'Fletchling', 'Type 1'] = 'normal'
pokemon_set.loc[pokemon_set['Name'] == 'Fletchling', 'Type 2'] = 'flying'

names = ['Dewgong', 'Magnemite', 'Magneton', 'Magnezone', 'Omanyte', 'Omastar', 'Kabuto', 'Kabutops', 'Scizor', 'Empoleon', 'Spiritomb', 'Ferrothorn']
pokemon_set.loc[pokemon_set['Name'].isin(names), 'Type 1'], pokemon_set.loc[pokemon_set['Name'].isin(names), 'Type 2'] = pokemon_set.loc[pokemon_set['Name'].isin(names), 'Type 2'], pokemon_set.loc[pokemon_set['Name'].isin(names), 'Type 1']

Filter by progress

In [115]:
def read_routes():
    with open('data/routes.csv', 'r') as f:
        routext = f.read().split('\n')
    routes = pd.DataFrame(columns=['Medals', 'Locations'])
    for r in routext:
        if r != '':
            routes.loc[len(routes)] = [len(routes), [i.title() for i in r.split(',')]]
    return routes

def filter_encounters_by_medals(encounters):
    if medals < 2:
        encounters = encounters.loc[~encounters['Location'].str.contains('OldRod')]
    if medals < 4:
        encounters = encounters.loc[~encounters['Location'].str.contains('GoodRod')]
    if medals < 6:
        encounters = encounters.loc[~encounters['Location'].str.contains('SuperRod')]
        encounters = encounters.loc[~encounters['Location'].str.contains('Water')]
    if medals < 10:
        encounters = encounters.loc[~encounters['Location'].str.contains('Underwater')]
    return encounters

def assign_medals_to_pokemon(df, routes, encounters):
    df['Route'] = np.nan
    for i in range(len(routes)):
        route_pokemon = routes['Locations'][i]
        pokes = encounters.loc[encounters['Name'].isin(route_pokemon)]['Pokemon'].values
        pokes = [item for sublist in pokes for item in sublist]
        df.loc[(df['Name'].isin(pokes)) & (df['Route'].isnull()), 'Route'] = i
    return df

def add_traded_pokemon(df):
    traded_pokemon = {
        0: ["Bellsprout", "Tirogue", "Pineco"],
        1: ["Charmander", "Squirtle", "Bulbasaur", "Yanma", "Wooper"],
        2: ["Poliwag", "Farfetch'D", "Doduo", "Exeggcute", "Seel", "Growlithe", "Mr. Mime", "Stantler", "Mareep", "Lileep", "Anorith"],
        3: ["Ponyta", "Eevee", "Porygon", "Shieldon", "Cranidos", "Wynaut", "Honedge"],
        4: ["Gligar", "Heracross", "Totodile", "Chikorita", "Cyndaquil"],
        5: ["Smeargle", "Hitmonlee", "Hitmonchan", "Hitmontop", "Lapras"],
        6: ["Skarmory", "Seel", "Electrode", "Chansey", "Larvitar"],
        7: ["Sneasel"],
        8: ["Mudkip", "Treecko", "Torchic", "Ralts", "Beldum", "Lucario", "Gible", "Bidoof", "Chimchar", "Piplup", "Turtwig", "Pawniard"]
    }
    df2 = df.loc[(df['Route'].notnull()) & (df['Route'] <= medals)]
    for i in range(len(traded_pokemon)):
        if medals >= i:
            for p in traded_pokemon[i]:
                if p in df['Name'].values:
                    df2 = df2.append(df.loc[df['Name'] == p])

    return df2

def add_evolutions(df, dfcp):
    obedience_level = min(100, 10 + medals * 10)
    for i in range(len(df)):
        if df.iloc[i]['Evolution'] not in df['Name'].values:
            if df.iloc[i]['Min Level'] and type(df.iloc[i]['Min Level']) == int and df.iloc[i]['Min Level'] < obedience_level:
                df = df.append(dfcp.loc[dfcp['Name'] == df.iloc[i]['Evolution']])
    return df


def filter_medals(df):
    import warnings
    warnings.filterwarnings("ignore")
    # Read the files
    encounters = pd.read_json('data/encounters.json')
    evos = pd.read_json('data/pokemon_evolutions.json')
    routes = read_routes()

    df = pd.merge(df, evos, left_on='Name', right_on='Name', how='left')
    dfcp = df.copy()

    # Filter encounters by medals
    encounters = filter_encounters_by_medals(encounters)

    # Assign medals to pokemon
    df = assign_medals_to_pokemon(df, routes, encounters)
    # Add traded and gift pokemon
    df = add_traded_pokemon(df)
    # Add evolutions to the dataframe
    df = add_evolutions(df, dfcp)
    df.drop(columns=['Route'], inplace=True)

    return df


if filter_by_medals: pokemon_set = filter_medals(pokemon_set)
if filter_by_pokemon: pokemon_set = pokemon_set.loc[~pokemon_set['Name'].isin(unwanteds)]

Type related stuff

In [116]:
type_chart = {'normal': {'normal': 1, 'fighting': 1, 'flying': 1, 'poison': 1, 'ground': 1, 'rock': -2, 'bug': 1, 'ghost': -4, 'steel': -2, 'fire': 1, 'water': 1, 'grass': 1, 'electric': 1, 'psychic': 1, 'ice': 1, 'dragon': 1, 'dark': 1, 'fairy': 1},
            'fighting': {'normal': 2, 'fighting': 1, 'flying': -2, 'poison': -2, 'ground': 1, 'rock': 2, 'bug': -2, 'ghost': -4, 'steel': 2, 'fire': 1, 'water': 1, 'grass': 1, 'electric': 1, 'psychic': -2, 'ice': 2, 'dragon': 1, 'dark': 2, 'fairy': -2},
            'flying': {'normal': 1, 'fighting': 2, 'flying': 1, 'poison': 1, 'ground': 1, 'rock': -2, 'bug': 2, 'ghost': 1, 'steel': -2, 'fire': 1, 'water': 1, 'grass': 2, 'electric': -2, 'psychic': 1, 'ice': 1, 'dragon': 1, 'dark': 1, 'fairy': 1},
            'poison': {'normal': 1, 'fighting': 1, 'flying': 1, 'poison': -2, 'ground': -2, 'rock': -2, 'bug': 1, 'ghost': -2, 'steel': -4, 'fire': 1, 'water': 1, 'grass': 2, 'electric': 1, 'psychic': 1, 'ice': 1, 'dragon': 1, 'dark': 1, 'fairy': 2},
            'ground': {'normal': 1, 'fighting': 1, 'flying': -4, 'poison': 2, 'ground': 1, 'rock': 2, 'bug': -2, 'ghost': 1, 'steel': 2, 'fire': 2, 'water': 1, 'grass': -2, 'electric': 2, 'psychic': 1, 'ice': 1, 'dragon': 1, 'dark': 1, 'fairy': 1},
            'rock': {'normal': 1, 'fighting': -2, 'flying': 2, 'poison': 1, 'ground': -2, 'rock': 1, 'bug': 2, 'ghost': 1, 'steel': -2, 'fire': 2, 'water': 1, 'grass': 1, 'electric': 1, 'psychic': 1, 'ice': 2, 'dragon': 1, 'dark': 1, 'fairy': 1},
            'bug': {'normal': 1, 'fighting': -2, 'flying': -2, 'poison': -2, 'ground': 1, 'rock': 1, 'bug': 1, 'ghost': -2, 'steel': -2, 'fire': -2, 'water': 1, 'grass': 2, 'electric': 1, 'psychic': 2, 'ice': 1, 'dragon': 1, 'dark': 2, 'fairy': -2},
            'ghost': {'normal': -4, 'fighting': 1, 'flying': 1, 'poison': 1, 'ground': 1, 'rock': 1, 'bug': 1, 'ghost': 2, 'steel': 1, 'fire': 1, 'water': 1, 'grass': 1, 'electric': 1, 'psychic': 2, 'ice': 1, 'dragon': 1, 'dark': -2, 'fairy': 1},
            'steel': {'normal': 1, 'fighting': 1, 'flying': 1, 'poison': 1, 'ground': 1, 'rock': 2, 'bug': 1, 'ghost': 1, 'steel': -2, 'fire': -2, 'water': -2, 'grass': 1, 'electric': -2, 'psychic': -2, 'ice': 2, 'dragon': 1, 'dark': 1, 'fairy': 2},
            'fire': {'normal': 1, 'fighting': 1, 'flying': 1, 'poison': 1, 'ground': 1, 'rock': -2, 'bug': 2, 'ghost': 1, 'steel': 2, 'fire': -2, 'water': -2, 'grass': 2, 'electric': 1, 'psychic': 1, 'ice': 2, 'dragon': -2, 'dark': 1, 'fairy': 1},
            'water': {'normal': 1, 'fighting': 1, 'flying': 1, 'poison': 1, 'ground': 2, 'rock': 2, 'bug': 1, 'ghost': 1, 'steel': 1, 'fire': 2, 'water': -2, 'grass': -2, 'electric': 1, 'psychic': 1, 'ice': 1, 'dragon': -2, 'dark': 1, 'fairy': 1},
            'grass': {'normal': 1, 'fighting': 1, 'flying': -2, 'poison': -2, 'ground': 2, 'rock': 2, 'bug': -2, 'ghost': 1, 'steel': -2, 'fire': -2, 'water': 2, 'grass': -2, 'electric': 1, 'psychic': 1, 'ice': 1, 'dragon': -2, 'dark': 1, 'fairy': 1},
            'electric': {'normal': 1, 'fighting': 1, 'flying': 2, 'poison': 1, 'ground': -4, 'rock': 1, 'bug': 1, 'ghost': 1, 'steel': 1, 'fire': 1, 'water': 2, 'grass': -2, 'electric': -2, 'psychic': 1, 'ice': 1, 'dragon': -2, 'dark': 1, 'fairy': 1},
            'psychic': {'normal': 1, 'fighting': 2, 'flying': 1, 'poison': 2, 'ground': 1, 'rock': 1, 'bug': 1, 'ghost': 1, 'steel': -2, 'fire': 1, 'water': 1, 'grass': 1, 'electric': 1, 'psychic': -2, 'ice': 1, 'dragon': 1, 'dark': -4, 'fairy': 1},
            'ice': {'normal': 1, 'fighting': 1, 'flying': 2, 'poison': 1, 'ground': 2, 'rock': 1, 'bug': 1, 'ghost': 1, 'steel': -2, 'fire': -2, 'water': -2, 'grass': 2, 'electric': 1, 'psychic': 1, 'ice': -2, 'dragon': 2, 'dark': 1, 'fairy': 1},
            'dragon': {'normal': 1, 'fighting': 1, 'flying': 1, 'poison': 1, 'ground': 1, 'rock': 1, 'bug': 1, 'ghost': 1, 'steel': -2, 'fire': 1, 'water': 1, 'grass': 1, 'electric': 1, 'psychic': 1, 'ice': 1, 'dragon': 2, 'dark': 1, 'fairy': -4},
            'dark': {'normal': 1, 'fighting': -2, 'flying': 1, 'poison': 1, 'ground': 1, 'rock': 1, 'bug': 1, 'ghost': 2, 'steel': 1, 'fire': 1, 'water': 1, 'grass': 1, 'electric': 1, 'psychic': 2, 'ice': 1, 'dragon': 1, 'dark': -2, 'fairy': -2},
            'fairy': {'normal': 1, 'fighting': 2, 'flying': 1, 'poison': -2, 'ground': 1, 'rock': 1, 'bug': 1, 'ghost': 1, 'steel': -2, 'fire': -2, 'water': 1, 'grass': 1, 'electric': 1, 'psychic': 1, 'ice': 1, 'dragon': 2, 'dark': 2, 'fairy': 1}}

def calculate_multiplier(type, value, split=0):
    split_b = physical_boost if split == 0 else special_boost
    if type in boosted_types:
        return value * boost * split_b
    elif type in nerfed_types:
        return value * nerf * split_b
    return value * split_b

def get_defensive_score(type1, type2):
    types = [type1, int(type2)] if not type(type2) == np.str_ else [type1]
    score = 0
    for attacking_type, value in type_chart.items():
        local_scores = [0 if type(value[defending_type]) == float or abs(value[defending_type]) < 2 else value[defending_type] for defending_type in types]
        if 4 in local_scores or -4 in local_scores: score += 4
        elif not(sum(local_scores) == 0): score += -sum([calculate_multiplier(attacking_type, score, 0) if attacking_type in physical_types else calculate_multiplier(attacking_type, score, 1) for score in local_scores])
    return score

def get_offensive_score(type1, type2):
    def eval_type(offensive_type):
        return np.sum([calculate_multiplier(defending_type, value) if type(value) != np.str_ and value != 0 else -4 for defending_type, value in type_chart[offensive_type].items()])
    return eval_type(type1) + eval_type(type2) if type(type2) != np.str_ else eval_type(type1)


def determine_fusion_types(types1, types2):
    types = []
    for i in range(len(types1)):
        type1_1, type1_2 = types1[i]; type2_1, type2_2 = types2[i]
        if type(type1_2) == float: type1_2 = type1_1
        if type(type2_2) == float: type2_2 = type2_1

        if type1_1 == type2_2: type2_2 = np.nan
        if type2_1 == type1_2: type1_2 = np.nan

        types.append([[type1_1, type2_2], [type2_1, type1_2]])
    return np.array(types)

Rating

In [117]:
def calc_fusion_stats(stats_pairs):
    stats1 = stats_pairs[:, 0, :]
    stats2 = stats_pairs[:, 1, :]
    fusion1 = np.array([stats1[:, 0] * 2/3 + stats2[:, 0] * 1/3,
                        stats1[:, 1] * 1/3 + stats2[:, 1] * 2/3,
                        stats1[:, 2] * 1/3 + stats2[:, 2] * 2/3,
                        stats1[:, 3] * 2/3 + stats2[:, 3] * 1/3,
                        stats1[:, 4] * 2/3 + stats2[:, 4] * 1/3,
                        stats1[:, 5] * 1/3 + stats2[:, 5] * 2/3]).T.astype(int)
    
    fusion2 = np.array([stats1[:, 0] * 1/3 + stats2[:, 0] * 2/3,
                        stats1[:, 1] * 2/3 + stats2[:, 1] * 1/3,
                        stats1[:, 2] * 2/3 + stats2[:, 2] * 1/3,
                        stats1[:, 3] * 1/3 + stats2[:, 3] * 2/3,
                        stats1[:, 4] * 1/3 + stats2[:, 4] * 2/3,
                        stats1[:, 5] * 2/3 + stats2[:, 5] * 1/3]).T.astype(int)
    return np.stack([fusion1, fusion2], axis=1)

def rate_pokemon(stats, multipliers, types, off_mul, def_mul):
    defensive_score = get_defensive_score(types[0], types[1]) / 20 * def_mul
    offensive_score = get_offensive_score(types[0], types[1]) / 10 * off_mul
    stat_score = (stats[0] * multipliers[0] + stats[1] * multipliers[1] + stats[2] * multipliers[2] + stats[3] * multipliers[3] + stats[4] * multipliers[4] + stats[5] * multipliers[5])  / 1000 * (1 - off_mul - def_mul)
    
    return (stat_score + defensive_score + offensive_score) * 30

def rate_fusion(stats, stat_multipliers, fusion_types, off_mul, def_mul):
    return [rate_pokemon(statn, stat_multipliers, fusion_typen, off_mul, def_mul) for statn, fusion_typen in zip(stats, fusion_types)]

### Main algorithm

A bit confusing, works with data in parallel.

In [118]:
def find_best_fusion(profile, dataset, cap, force_pokemon=None):
    pokemon = dataset[["Name", "Total", "Type 1", "Type 2", "HP", "Attack", "Defense", "Special Attack", "Special Defense", "Speed"]]
    pokemon = pokemon[pokemon["Total"] < cap]
    name_type_cols = ["Name", "Type 1", "Type 2"]
    stats_cols = ["HP", "Attack", "Defense", "Special Attack", "Special Defense", "Speed"]
    pairs = np.array(list(combinations(pokemon[name_type_cols + stats_cols].values, 2)))
    if force_pokemon:
        pairs = pairs[(pairs[:, 0, 0] == force_pokemon) | (pairs[:, 1, 0] == force_pokemon)]

    fusion_stats = calc_fusion_stats(pairs[:, :, 3:])
    fusion_types = determine_fusion_types(pairs[:, 0, 1:3], pairs[:, 1, 1:3])
    fusion_scores = rate_fusion(fusion_stats.reshape(-1, 6), profile[0], fusion_types.reshape(-1, 2), profile[1], profile[2])
    fusion_scores = np.array(fusion_scores).reshape(-1, 2)

    max_index = np.unravel_index(fusion_scores.argmax(), fusion_scores.shape)
    best_fusion_history = (pairs[max_index[0], 0, 0], pairs[max_index[0], 1, 0], fusion_scores[max_index], fusion_stats[max_index[0], max_index[1]], fusion_types[max_index[0], max_index[1]])

    return [best_fusion_history]

### RESULTS

In [125]:
res = ""
stat_names = ["HP", "Attack", "Defense", "Special Attack", "Special Defense", "Speed"]
stat_dict = {stat_names[i]: result[3][i] for i in range(len(stat_names))}


for profile_name, profile in profiles.items():
    res += f"Results for {profile_name}: \n"
    print(profile_name)

    if profile[0][2] >= profile[0][4]: 
        physical_boost = (profile[0][2] - profile[0][4]) / 10 + 1
        special_boost = 1
    else: 
        special_boost = (profile[0][4] - profile[0][2]) / 10 + 1
        physical_boost = 1

    partner = partner if force_partner else None
    results = list(reversed(find_best_fusion(profile, pokemon_set, stat_cap, partner)[-5:]))
    
    for result in results:
        lc = f"{result[0]} + {result[1]}: {round(result[2], 2)}\n" + f"Stats: {', '.join(f'{name}: {stat}' for name, stat in zip(stat_names, result[3]))}\n" + f"Types: {', '.join(x.title() for x in result[4])}\n\n"
        res += lc
        print(lc)

with open("results.txt", "w") as f:
    f.write(res)

Fast physical attacker
Dugtrio + Kingler: 60.65
Stats: HP: 41, Attack: 113, Defense: 93, Special Attack: 50, Special Defense: 63, Speed: 90
Types: Ground, Water


Fast special attacker
Persian + Kadabra: 56.85
Stats: HP: 48, Attack: 58, Defense: 50, Special Attack: 101, Special Defense: 68, Speed: 111
Types: Psychic, Normal


Physical tank
Onix + Quagsire: 50.53
Stats: HP: 75, Attack: 58, Defense: 135, Special Attack: 53, Special Defense: 58, Speed: 58
Types: Water, Rock


Special tank
Hypno + Noctowl: 42.82
Stats: HP: 95, Attack: 65, Defense: 63, Special Attack: 75, Special Defense: 102, Speed: 68
Types: Flying, Psychic


Hybrid tank
Onix + Noctowl: 42.48
Stats: HP: 78, Attack: 46, Defense: 123, Special Attack: 60, Special Defense: 79, Speed: 70
Types: Flying, Rock


Balanced overall
Kingler + Noctowl: 48.68
Stats: HP: 85, Attack: 103, Defense: 93, Special Attack: 67, Special Defense: 80, Speed: 73
Types: Flying, Water


Physical wallbreaker
Sandslash + Kingler: 58.4
Stats: HP: 68, At